# Using `pyfwg` for direct morphing with `morph_epw_global` and `morph_epw_europe` (v4 Update)

This notebook demonstrates the simplest and most direct way to use the `pyfwg` library: the `morph_epw_global` and `morph_epw_europe` functions.

These functions are designed as one-shot tools that provide full access to all of the Future Weather Generator's parameters without requiring the user to manage the multi-step `MorphingWorkflow` class. It handles parameter validation, temporary file management, and final file placement in a single call.

**Note:** `pyfwg` now fully supports **FutureWeatherGenerator v4.0.x** (Global) and **v2.0.x** (Europe), which include a new keyword-based CLI, support for string-based parameter options (like `'AVG4P'` for interpolation), and additional output formats (like `'MET'` or `'CSV'`).

## Understanding the Parameters

The `morph_epw_global` and `morph_epw_europe` functions are powerful because these exposes all of the underlying tool's options. Here is a complete reference for all their arguments.

### Core Parameters

*   `epw_paths` (`Union[str, List[str]]`): **Required.** The path to a single EPW file (as a string) or a list of paths to multiple EPW files.
*   `fwg_jar_path` (`str`): **Required.** The absolute or relative path to the Future Weather Generator `.jar` file (e.g., `FutureWeatherGenerator_v4.0.2.jar`).

### Workflow Control Parameters

*   `output_dir` (`str`, default: `'./morphed_epws'`): The directory where the final morphed files will be saved.
*   `delete_temp_files` (`bool`, default: `True`): If `True`, the temporary folders are deleted after processing. Set to `False` for debugging.
*   `temp_base_dir` (`str`, default: `'./morphing_temp_results'`): The base directory where temporary folders will be created.
*   `fwg_show_tool_output` (`bool`, default: `False`): If `True`, prints the FWG tool's console output in real-time. Highly recommended for monitoring progress.
*   `fwg_params` (`Optional[Dict]`, default: `None`): A dictionary for providing a base set of FWG parameters. Any explicit `fwg_` argument will override the values in this dictionary.

### Future Weather Generator Tool Parameters

These arguments correspond directly to the options in the FWG command-line tool.

*   `fwg_gcms` (`Optional[List[str]]`, default: `None`): **Only valid for `morph_epw_global`**. A list of GCMs to use. If `None`, the full default list is used to create an ensemble.
*   `fwg_rcm_pairs` (`Optional[List[str]]`, default: `None`): **Only valid for `morph_epw_europe`**. List of GCM-RCM pairs to use. If `None`, the tool's default list is used.
*   `fwg_create_ensemble` (`bool`, default: `True`): If `True`, creates an ensemble from the selected GCMs.
*   `fwg_winter_sd_shift` (`float`, default: `0.0`): Temperature's standard deviation shift for the winter peak month. Range: -2.0 to 2.0.
*   `fwg_summer_sd_shift` (`float`, default: `0.0`): Temperature's standard deviation shift for the summer peak month. Range: -2.0 to 2.0.
*   `fwg_month_transition_hours` (`int`, default: `72`): Number of hours to smooth the transition between months. Range: 0 to 336.
*   `fwg_use_multithreading` (`bool`, default: `True`): If `True`, performs computations in multithread.
*   `fwg_interpolation_method_id` (`Union[int, str]`, default: `0`): Selects the grid method. **Legacy options:** `0` (IDW), `1` (AVG4P), `2` (NP). **V4/Europe v2 options:** you can use strings: `'IDW'`, `'BI'`, `'AVG4P'`, `'NP'`.
*   `fwg_limit_variables` (`bool`, default: `True`): If `True`, bounds each generated variable to its physical limits.
*   `fwg_solar_hour_adjustment` (`Union[int, str]`, default: `1`): Option for solar hour adjustment. **Legacy options:** `0`, `1`, `2`. **V4 options:** `'None'`, `'By_Month'`, `'By_Day'`.
*   `fwg_diffuse_irradiation_model` (`Union[int, str]`, default: `1`): Option for diffuse solar irradiation model. **Legacy options:** `0`, `1`, `2`. **V4 options:** `'Ridley_Boland_Lauret_2010'`, `'Engerer_2015'`, `'Paulescu_Blaga_2019'`.
*   `fwg_add_uhi` (`bool`, default: `True`): Option to pre-process the Urban Heat Island (UHI) effect.
*   `fwg_epw_original_lcz` (`int`, default: `14`): The Local Climate Zone (LCZ) of the original EPW. Range: 1 to 17.
*   `fwg_target_uhi_lcz` (`int`, default: `1`): The target LCZ of the generated EPW. Range: 1 to 17.
*   `fwg_output_type` (`str`, default: `'EPW'`): **New in V4/Europe v2.** Format of output files: `'EPW'`, `'SPAIN_MET'`, or `'PORTUGAL_CSV'`.
*   `fwg_version` (`Optional[str]`, default: `None`): Force a specific CLI version (e.g., `'4'`). If `None`, `pyfwg` auto-detects from the JAR filename.

## Using `morph_epw_global`

### Step 1: Imports and Setup

First, we'll import the necessary function and set up our file paths. 

**Important:** You must change the `jar_path` variable to the correct location of your Future Weather Generator `.jar` file. `pyfwg` will automatically detect the version (v3 or v4) and adapt the CLI command accordingly.

In [1]:
import os
from pyfwg import get_fwg_parameters_info, morph_epw_global, get_available_lczs, check_lcz_availability, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# Define the path to the EPW file to be processed. You can also define a list of paths to process multiple EPW files.
epw_file = 'epws/w_pattern/sevilla_uhi-type-1.epw'

# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar"

### Exploring FWG Parameters

Before configuring the workflow, you might want to know which parameters are available and what values they accept. This is especially useful for the new string-based options in V4.

You can use the `get_fwg_parameters_info` utility to get a detailed list of all parameters.

In [2]:
import json
# Get the info and print it nicely
info = get_fwg_parameters_info()
print(json.dumps(info, indent=4))

{
    "fwg_gcms": {
        "description": "List of Global Climate Models (GCMs) to use for the Global tool.",
        "allowed_values": [
            "BCC_CSM2_MR",
            "CAS_ESM2_0",
            "CMCC_ESM2",
            "CNRM_CM6_1",
            "CNRM_CM6_1_HR",
            "CNRM_ESM2_1",
            "CanESM5",
            "CanESM5_1",
            "CanESM5_CanOE",
            "EC_Earth3",
            "EC_Earth3_Veg",
            "EC_Earth3_Veg_LR",
            "FGOALS_g3",
            "GFDL_ESM4",
            "GISS_E2_1_G",
            "GISS_E2_1_H",
            "GISS_E2_2_G",
            "IPSL_CM6A_LR",
            "MIROC6",
            "MIROC_ES2H",
            "MIROC_ES2L",
            "MRI_ESM2_0",
            "UKESM1_0_LL"
        ],
        "default": "All available GCMs",
        "applies_to": "Global"
    },
    "fwg_rcm_pairs": {
        "description": "List of GCM-RCM model pairs to use for the Europe-specific tool.",
        "allowed_values": [
            "CNRM_CER

### Step 2: Pre-flight Checks (Recommended). Check Available LCZs.

Before running a long morphing process, it's a good practice to validate your parameters. `pyfwg` provides utility functions to help with this.

The `fwg_epw_original_lcz` and `fwg_target_uhi_lcz` parameters depend on which Local Climate Zones are available for a specific weather file location. We can find this out using `get_available_lczs`.

In [3]:
available_zones = get_available_lczs(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    # In java_class_path_prefix, use 'futureweathergenerator' for the global scope FWG tool, otherwise 'futureweathergenerator_europe'
    java_class_path_prefix='futureweathergenerator',
    show_tool_output=False,
)

2025-12-30 20:28:04 - INFO - --- Fetching available LCZs for 1 EPW file(s) ---
2025-12-30 20:28:04 - INFO - Checking LCZ pair (Original: 0, Target: 0) availability for sevilla_uhi-type-1.epw...
2025-12-30 20:28:05 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-12-30 20:28:05 - INFO - Executing command: java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -u -epw=C:\Users\sanga\AppData\Local\Temp\tmpm08u8xr7\sevilla_uhi-type-1.epw -output_folder=C:\Users\sanga\AppData\Local\Temp\tmpm08u8xr7\ -uhi=true:0:0 -output_type=EPW
2025-12-30 20:28:23 - INFO - Available LCZs for 'sevilla_uhi-type-1.epw': [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


In [4]:
# Print the results in a user-friendly format
for filename, lcz_list in available_zones.items():
    print(f"\nAvailable LCZs for: {filename}")
    if lcz_list:
        print(f"  {lcz_list}")
    else:
        print("  Could not be determined.")


Available LCZs for: sevilla_uhi-type-1.epw
  [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


### Step 3: Run the Morphing Process

Now, we can call the `morph_epw_global` function. We will override a few of the default parameters to demonstrate how to customize the process. To speed up the process, let's use just one GCM. If you don't know which are these, you can import the list containing the default models:

In [5]:
from pyfwg import get_fwg_parameters_info, DEFAULT_GLOBAL_GCMS
print(DEFAULT_GLOBAL_GCMS)

{'GISS_E2_1_H', 'CNRM_CM6_1', 'MIROC6', 'IPSL_CM6A_LR', 'CMCC_ESM2', 'EC_Earth3_Veg', 'EC_Earth3_Veg_LR', 'BCC_CSM2_MR', 'CanESM5_CanOE', 'FGOALS_g3', 'GISS_E2_2_G', 'CanESM5_1', 'GFDL_ESM4', 'CNRM_CM6_1_HR', 'MRI_ESM2_0', 'MIROC_ES2H', 'EC_Earth3', 'MIROC_ES2L', 'CanESM5', 'UKESM1_0_LL', 'GISS_E2_1_G', 'CAS_ESM2_0', 'CNRM_ESM2_1'}


So let's use for instance "CanESM5". Note that in **V4**, we can use descriptive strings for parameters like `fwg_interpolation_method_id`.

In [6]:
# The validation of parameters is executed automatically inside the function.
created_files_custom = morph_epw_global(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    output_dir='./custom_output_global',
    fwg_show_tool_output=True,      # See the tool's progress in real-time
    fwg_gcms=['CanESM5'],           # Use only one specific GCM for speed
    fwg_interpolation_method_id='AVG4P',  # V4 feature: Use string names!
    fwg_epw_original_lcz=2,         # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3,          # Use a validated LCZ from the check above
    fwg_version='4'                 # Explicitly force V4 syntax
)

2025-12-30 20:28:24 - INFO - --- Starting Direct Global Morphing Process ---
2025-12-30 20:28:24 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-12-30 20:28:24 - INFO - Validating LCZ availability for sevilla_uhi-type-1.epw...
2025-12-30 20:28:24 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for sevilla_uhi-type-1.epw...
2025-12-30 20:28:24 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global
  - EPWs to be Morphed (1 files):
    - sevilla_uhi-type-1.epw

  For input file: sevilla_uhi-type-1.epw
    -> Generated 'ssp126_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp126_2050.epw
    -> Generated 'ssp245_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp245_2050.epw
    -> Generated 'ssp370_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp370_2050.epw
    -> Generated 'ssp585_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp585_2050.epw
    -> Generated 'ssp126_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp126_2080.epw
    -> Generated 'ssp245_2080.epw' will be moved to: D

2025-12-30 20:28:24 - INFO - Executing command: java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -u -epw=C:\Users\sanga\AppData\Local\Temp\tmp6c1xil_q\sevilla_uhi-type-1.epw -output_folder=C:\Users\sanga\AppData\Local\Temp\tmp6c1xil_q\ -uhi=true:2:3 -output_type=EPW
2025-12-30 20:28:40 - INFO - UHI effect applied successfully.
2025-12-30 20:28:40 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-12-30 20:28:40 - INFO - Copied input file to temporary directory: ./morphing_temp_results\sevilla_uhi-type-1\sevilla_uhi-type-1.epw



-------------------- Executing FWG for sevilla_uhi-type-1.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw -output_folder=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\sevilla_uhi-type-1\ -models=CanESM5 -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=AVG4P -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=EPW
  --- FWG Real-time Output ---


2025-12-30 20:29:00 - INFO - Direct global morphing complete. 18 files created in D:\Python\pyfwg\pyfwg\tutorials\custom_output_global


  --- End of FWG Output ---


In [7]:
print("\nSuccessfully created files:")
for f in created_files_custom:
    print(f)


Successfully created files:
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp245_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp245_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp245_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Origina

### Expected Output

When you run the code above with **V4**, you should see:
1. A detailed printout of the **keyword-based command** being executed (e.g., `-epw=... -models=CanESM5`).
2. The real-time console output from the Future Weather Generator tool.
3. A final message listing all the created files in the `./custom_output_global` directory.

Let's delete the files we just generated.

In [8]:
import shutil
if os.path.exists('custom_output_global'):
    shutil.rmtree('custom_output_global')

## Using `morph_epw_europe`

>**FYI**: the usage of `morph_epw_europe` is very similar to the global version. The only differences are:

>* Instead of argument `fwg_gcms`, you need to use `fwg_rcm_pairs`.
>* Instead of variable `DEFAULT_GLOBAL_GCMS`, the available models are in variable `DEFAULT_EUROPE_RCMS`.
>* Future scenarios are RCP instead of SSP.
>* You have to remember using the `jar_path` to the FutureWeatherGenerator_Europe_vx.x.x.jar file.

### Step 1: Imports and Setup

**Important:** Change the `jar_path` variable to the correct location of your European version JAR (v1.x for legacy or v2.x for the update).

In [9]:
from pyfwg import get_fwg_parameters_info, morph_epw_europe

# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path_europe = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar"

### Step 2: Run the Morphing Process

Now, we can call the `morph_epw_europe` function. In **Europe v2**, we can also specify a different output format like `'MET'`.

In [10]:
from pyfwg import get_fwg_parameters_info, DEFAULT_EUROPE_RCMS

created_files_custom = morph_epw_europe(
    epw_paths=epw_file,
    fwg_jar_path=jar_path_europe,
    output_dir='./custom_output_europe',
    fwg_show_tool_output=True,
    fwg_rcm_pairs=[list(DEFAULT_EUROPE_RCMS)[0]],  # Use the first pair for speed
    fwg_interpolation_method_id='NP',              # Nearest Point
    fwg_epw_original_lcz=2,
    fwg_target_uhi_lcz=3,
    fwg_output_type='SPAIN_MET'                         # Output formatted as MET files
)

2025-12-30 20:29:00 - INFO - --- Starting Europe-Specific Direct Morphing Process ---
2025-12-30 20:29:00 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-12-30 20:29:00 - INFO - Auto-detected FWG version: 2
2025-12-30 20:29:00 - INFO - Validating LCZ availability for sevilla_uhi-type-1.epw...
2025-12-30 20:29:00 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for sevilla_uhi-type-1.epw...
2025-12-30 20:29:00 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe
  - EPWs to be Morphed (1 files):
    - sevilla_uhi-type-1.epw

  For input file: sevilla_uhi-type-1.epw
    -> Generated 'rcp26_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp26_2050.epw
    -> Generated 'rcp45_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp45_2050.epw
    -> Generated 'rcp85_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp85_2050.epw
    -> Generated 'rcp26_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp26_2080.epw
    -> Generated 'rcp45_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp45_2080.epw
    -> Generated 'rcp85_2080.epw' will be moved to: D:\Py

2025-12-30 20:29:00 - INFO - Executing command: java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar" -u -epw=C:\Users\sanga\AppData\Local\Temp\tmp4znojmmf\sevilla_uhi-type-1.epw -output_folder=C:\Users\sanga\AppData\Local\Temp\tmp4znojmmf\ -uhi=true:2:3 -output_type=EPW
2025-12-30 20:29:15 - INFO - UHI effect applied successfully.
2025-12-30 20:29:15 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-12-30 20:29:15 - INFO - Copied input file to temporary directory: ./morphing_temp_results_europe\sevilla_uhi-type-1\sevilla_uhi-type-1.epw



-------------------- Executing FWG for sevilla_uhi-type-1.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw -output_folder=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\sevilla_uhi-type-1\ -models=MOHC_HadGEM2_ES_DMI_HIRHAM5 -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=NP -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=SPAIN_MET
  --- FWG Real-time Output ---


2025-12-30 20:29:53 - WARNING - PermissionError deleting ./morphing_temp_results_europe\sevilla_uhi-type-1. Retrying in 0.5s... (Attempt 1/5)


  --- End of FWG Output ---


2025-12-30 20:29:53 - WARNING - PermissionError deleting ./morphing_temp_results_europe\sevilla_uhi-type-1. Retrying in 0.5s... (Attempt 2/5)
2025-12-30 20:29:54 - INFO - Europe morphing complete. 7 files created in D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe


In [11]:
print("\nSuccessfully created files:")
for f in created_files_custom:
    print(f)


Successfully created files:
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp45_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp45_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp85_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp85_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\sevilla_uhi-type-1.epw


Let's delete the files we just generated.

In [12]:
if os.path.exists('custom_output_europe'):
    shutil.rmtree('custom_output_europe')